## Data Preparation

0. Required library and Import the Data

In [1]:
import pandas as pd
import numpy as np
import sklearn
import re
from sklearn.datasets import load_diabetes
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
pd.set_option("display.max_colwidth", 500)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from datetime import datetime, timedelta
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns

/opt/homebrew/lib/python3.10/site-packages/xgboost/compat.py:105: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
df = pd.read_csv('./Bangkok Traffy.csv')
df.drop_duplicates(inplace=True)

- Make 'type' as list

In [3]:
df['type'] = (
    df['type']
    .str.strip("{}")          # remove leading/trailing {}
    .str.split(",")           # split into list
)


In [4]:
df.loc[[289638,196012,142463], ['ticket_id','type','organization','comment','address','subdistrict']]

,ticket_id,type,organization,comment,address,subdistrict
289638,2023-7ZTRF9,"[สะพาน, ท่อระบายน้ำ, ถนน, ร้องเรียน, ความปลอดภัย]","เขตหนองแขม,ฝ่ายโยธา เขตหนองแขม",[ ทีมงานว่าที่ สส. รักชนก ] ได้รับเรื่องร้องเรียน เรื่อง เรียบถนนเส้นข้างโรงเรียนวัดหนองแขมทางยาวไปถึงสะพานไม้ มีท่อระบายน้ำเป็นหลุม ฝาท่อสูงเกินกว่าถนน ทำให้เกิดอันตราย อยากให้เอายางมะตอยมาราดเพื่อให้ถนนเสมอกันเรียบสูง,91 ถนนเลียบคลองภาษีเจริญฝั่งใต้ แขวงหนองแขม เขตหนองแขม กรุงเทพมหานคร 10160 ประเทศไทย,หนองแขม
196012,2023-TWGYXL,[],"เขตบางซื่อ,ฝ่ายเทศกิจ เขตบางซื่อ,ผอ.เขตบางซื่อ (นายเจษฎา)",ป้ายโฆษณาครับ,22 ถ. รัชดาภิเษก แขวง วงศ์สว่าง เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,วงศ์สว่าง
142463,2022-7QUGYN,[ถนน],"เขตหลักสี่,ผอ.เขตหลักสี่ (นางสมฤดี),กลุ่มกรุงเทพเหนือ (นางวันทนีย์ วัฒนะ)",ตรงซอยถนนหน้าสำนักงานเขตหลักสี่,117/6 แจ้งวัฒนะ แขวง ทุ่งสองห้อง เขต หลักสี่ กรุงเทพมหานคร 10210 ประเทศไทย,ทุ่งสองห้อง


- set ticker_id as index

In [5]:
df['type'].apply(type).unique()
df.set_index('ticket_id', inplace=True)



In [6]:
df.loc[['2023-7ZTRF9', '2023-TWGYXL']]


,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
ticket_id,,,,,,,,,,,,,,,
2023-7ZTRF9,"[สะพาน, ท่อระบายน้ำ, ถนน, ร้องเรียน, ความปลอดภัย]","เขตหนองแขม,ฝ่ายโยธา เขตหนองแขม",[ ทีมงานว่าที่ สส. รักชนก ] ได้รับเรื่องร้องเรียน เรื่อง เรียบถนนเส้นข้างโรงเรียนวัดหนองแขมทางยาวไปถึงสะพานไม้ มีท่อระบายน้ำเป็นหลุม ฝาท่อสูงเกินกว่าถนน ทำให้เกิดอันตราย อยากให้เอายางมะตอยมาราดเพื่อให้ถนนเสมอกันเรียบสูง,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-06/5a65cb9be7e1a69cd838e9e30812aa4bbaf0ef4c.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-06/1686119531922.jpeg,"100.33999,13.67570",91 ถนนเลียบคลองภาษีเจริญฝั่งใต้ แขวงหนองแขม เขตหนองแขม กรุงเทพมหานคร 10160 ประเทศไทย,หนองแขม,หนองแขม,กรุงเทพมหานคร,2023-06-06 08:08:14.457214+00,เสร็จสิ้น,4.0,0,2023-06-07 06:32:13.130699+00
2023-TWGYXL,[],"เขตบางซื่อ,ฝ่ายเทศกิจ เขตบางซื่อ,ผอ.เขตบางซื่อ (นายเจษฎา)",ป้ายโฆษณาครับ,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-01/5680383cc9125134b3102adb397d0d7f17360059.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-01/8b9b18cc01db9e81708ac8c5e54cafa3eb5a9a33.jpg,"100.53265,13.83065",22 ถ. รัชดาภิเษก แขวง วงศ์สว่าง เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,วงศ์สว่าง,บางซื่อ,กรุงเทพมหานคร,2023-01-11 15:33:43.676854+00,เสร็จสิ้น,NaN,0,2023-01-12 05:11:10.739687+00


In [7]:
[k in df.index for k in ['2023-7ZTRF9', '2023-TWGYXL']]
df.shape


(787026, 15)

In [8]:
df.loc['2023-7ZTRF9']

type                                                                                                                                                                                       [สะพาน, ท่อระบายน้ำ, ถนน, ร้องเรียน, ความปลอดภัย]
organization                                                                                                                                                                                                  เขตหนองแขม,ฝ่ายโยธา เขตหนองแขม
comment          [ ทีมงานว่าที่ สส. รักชนก ] ได้รับเรื่องร้องเรียน เรื่อง เรียบถนนเส้นข้างโรงเรียนวัดหนองแขมทางยาวไปถึงสะพานไม้ มีท่อระบายน้ำเป็นหลุม ฝาท่อสูงเกินกว่าถนน ทำให้เกิดอันตราย อยากให้เอายางมะตอยมาราดเพื่อให้ถนนเสมอกันเรียบสูง
photo                                                                                                                    https://storage.googleapis.com/traffy_public_bucket/attachment/2023-06/5a65cb9be7e1a69cd838e9e30812aa4bbaf0ef4c.jpg
photo_after                                         

- Drop column that won't be used

In [9]:
df = df[df['province'].str.contains('กรุงเทพ', na=False)]
df.drop(columns=['photo','photo_after','coords','province','star','state','count_reopen','last_activity'],inplace=True)
df.shape

(785656, 7)

In [10]:
df.iloc[1]['address']
print(df.loc['2024-4B6VD4'].address)

704 ถ. ประชาธิปก แขวงสมเด็จเจ้าพระยา เขตคลองสาน กรุงเทพมหานคร 10600 ประเทศไทย


In [11]:
df.iloc[2]

type                                                                          [สะพาน]
organization                                                                  เขตสาทร
comment                               สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12
address         191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย
subdistrict                                                                   ยานนาวา
district                                                                         สาทร
timestamp                                               2021-09-26 05:03:52.594898+00
Name: 2021-7XATFA, dtype: object

- Extract 'district' and 'subdistrict' from 'address'

In [12]:
def extract_subdistrict(address):
    if not isinstance(address, str):
        return None
    match = re.search(r'แขวง\s*([^\s]+)', address)
    if match:
        return match.group(1)
    return None

def extract_district(address):
    if not isinstance(address, str):
        return None
    match = re.search(r'เขต\s*([^\s]+)', address)
    if match:
        return match.group(1)
    return None
df=df.copy()

extracted_sub = df['address'].apply(extract_subdistrict)
extracted_dist = df['address'].apply(extract_district)

df['subdistrict'] = df['subdistrict'].fillna(extracted_sub)
df['district'] = df['district'].fillna(extracted_dist)

In [13]:
df['district'] = df['district'].apply(lambda x: np.nan if x in [None, [], ''] else x)
df['subdistrict'] = df['subdistrict'].apply(lambda x: np.nan if x in [None, [], ''] else x)
df.dropna(subset=['district','subdistrict'],inplace=True)
print(df.district.unique().tolist())

['บางซื่อ', 'ประเวศ', 'สาทร', 'ลาดพร้าว', 'ดุสิต', 'สายไหม', 'ราชเทวี', 'บางรัก', 'จตุจักร', 'บางแค', 'บางกอกน้อย', 'คลองเตย', 'ลาดกระบัง', 'หลักสี่', 'บางกะปิ', 'หนองจอก', 'วัฒนา', 'บางเขน', 'คลองสาน', 'ราษฎร์บูรณะ', 'ดินแดง', 'ธนบุรี', 'ปทุมวัน', 'คันนายาว', 'พระนคร', 'ตลิ่งชัน', 'ดอนเมือง', 'ห้วยขวาง', 'สะพานสูง', 'พญาไท', 'สวนหลวง', 'ภาษีเจริญ', 'บางขุนเทียน', 'บางคอแหลม', 'บางนา', 'ทุ่งครุ', 'จอมทอง', 'คลองสามวา', 'วังทองหลาง', 'บางกอกใหญ่', 'ยานนาวา', 'พระโขนง', 'บึงกุ่ม', 'หนองแขม', 'มีนบุรี', 'บางพลัด', 'บางบอน', 'สัมพันธวงศ์', 'ป้อมปราบศัตรูพ่าย', 'ทวีวัฒนา', 'บางกรวย', 'บางพลี', 'เมืองนนทบุรี', 'พระประแดง', 'ลำลูกกา', 'เมืองสมุทรสาคร', 'สามพราน', 'ป้อมปราบศัตรูพ่า', 'เมืองปทุมธานี']


In [14]:
valid_districts = [
    "พระนคร", "ดุสิต", "หนองจอก", "บางรัก", "บางเขน", "บางกะปิ",
    "ปทุมวัน", "ป้อมปราบศัตรูพ่าย", "พระโขนง", "มีนบุรี", "ลาดกระบัง",
    "ยานนาวา", "สัมพันธวงศ์", "พญาไท", "ธนบุรี", "บางกอกใหญ่",
    "ห้วยขวาง", "คลองสาน", "ตลิ่งชัน", "บางกอกน้อย", "บางขุนเทียน",
    "ภาษีเจริญ", "หนองแขม", "ราษฎร์บูรณะ", "บางพลัด", "ดินแดง",
    "บึงกุ่ม", "สาทร", "บางซื่อ", "จตุจักร", "บางคอแหลม", "ประเวศ",
    "คลองเตย", "สวนหลวง", "จอมทอง", "ดอนเมือง", "ราชเทวี",
    "ลาดพร้าว", "วัฒนา", "บางแค", "หลักสี่", "สายไหม", "คันนายาว",
    "สะพานสูง", "วังทองหลาง", "คลองสามวา", "บางนา", "ทวีวัฒนา",
    "ทุ่งครุ", "บางบอน"
]
df['district'] = df['district'].str.strip()
df = df[df['district'].isin(valid_districts)]

In [15]:
df.sample(3)

,type,organization,comment,address,subdistrict,district,timestamp
ticket_id,,,,,,,
2022-HNFLKC,[ท่อระบายน้ำ],"ยังไม่มีหน่วยงานรับผิดชอบ,เขตคลองสาน,ผอ.เขตคลองสาน (ว่าที่ ร.ต.สรวุฒิ),กลุ่มกรุงธนเหนือ (นายเฉลิมพล โชตินุชิต)",ฝาท่อยุบลงไป,583 ถ. เจริญนคร แขวง บางลำภูล่าง เขตคลองสาน กรุงเทพมหานคร 10600 ประเทศไทย,บางลำภูล่าง,คลองสาน,2022-06-30 01:52:26.780841+00
2022-AGWQ99,"[แสงสว่าง, ถนน]","สำนักโยธา,โยธาธิการ,ฝ่ายโยธา เขตตลิ่งชัน,การไฟฟ้านครหลวง เขตธนบุรี,การไฟฟ้านครหลวง เขตบางใหญ่,การไฟฟ้านครหลวง MEA,ผอ.เขตตลิ่งชัน (นางชดาวรรณ),กลุ่มกรุงธนเหนือ (นายเฉลิมพล โชตินุชิต),ศูนย์เครื่องมือกล ส่วนก่อสร้างและบูรณะ 2 สำนักงานก่อสร้างและบูรณะ (สกบ.สนย.),เขตตลิ่งชัน,Bangkok Smart Lighting (สำนักการโยธา กทม.),สำนักการโยธา กทม.",ไฟฟ้าถนนดับ,QFR6+RM6 ถนน สวนผัก แขวง ตลิ่งชัน เขตตลิ่งชัน กรุงเทพมหานคร 10170 ประเทศไทย,ตลิ่งชัน,ตลิ่งชัน,2022-06-22 05:01:43.51958+00
KFM9JV,[ถนน],"เขตบางขุนเทียน,ฝ่ายเทศกิจ เขตบางขุนเทียน,ฝ่ายโยธา เขตบางขุนเทียน",ปัญหา: ต้องการให้นำสิ่งก่อสร้างที่ปิดกั้นภายในซอยออก เนื่องจาก มีการนำสิ่งก่อสร้างมาปิดกั้นถนนภายในซอยซึ่งเป็นพื้นที่สาธารณะ ทำให้เป็นซอยตัน ทั้งๆที่ซอยดังกล่าวสามารถทะลุกันได้ เคยแจ้งเจ้าหน้าที่ไปแล้ว แต่ยังไม่มีการดำเนินการแก้ไข\nประชาชนยินดีให้ติดต่อกลับ แต่ไม่เปิดเผยชื่อ เบอร์โทรแก่คู่กรณี\nซอย: พระราม2 ซอย 54 แยก 4 -2 - 1\nถนน: พระรามที่ 2\nจุดสังเกต: อยู่ภายในซอยดังกล่าว จุดสังเกต อยู่หลังแฟลตข้าราชการกรุงเทพมหานคร \nเขต: บางขุนเทียน\r\n#1555,43/126 ซอย เทียนทอง 7 แขวงแสมดำ เขตบางขุนเทียน กรุงเทพมหานคร 10150 ประเทศไทย,แสมดำ,บางขุนเทียน,2024-01-11 05:37:25.754734+00


In [16]:
df.timestamp = pd.to_datetime(df.timestamp, format='mixed')
df['timestamp_thai'] = df.timestamp.dt.tz_convert('Asia/Bangkok')

df['year'] = df['timestamp_thai'].dt.year
df['month'] = df['timestamp_thai'].dt.month
df['day'] = df['timestamp_thai'].dt.day

df.drop(columns=['timestamp','timestamp_thai'],inplace=True)

In [17]:
df.sample(3)

,type,organization,comment,address,subdistrict,district,year,month,day
ticket_id,,,,,,,,,
2023-P3N4TD,[ถนน],เขตลาดกระบัง,ถนนชำรุด,99/177 ซอย คุ้มเกล้า 11 แขวง ลำปลาทิว เขตลาดกระบัง กรุงเทพมหานคร 10520 ประเทศไทย,ลำปลาทิว,ลาดกระบัง,2023,5,18
2023-DMUN7Z,[ถนน],"เขตวัฒนา,ฝ่ายโยธา เขตวัฒนา",ถนนเป็นหลุมบ่อ เสื่อมสภาพ,287 ซอย สุขุมวิท 49/14 แขวงคลองตันเหนือ เขตวัฒนา กรุงเทพมหานคร 10110 ประเทศไทย,คลองตันเหนือ,วัฒนา,2023,9,5
GVV8AE,[แสงสว่าง],"เขตสายไหม,การไฟฟ้านครหลวง เขตบางเขน,การไฟฟ้านครหลวง MEA,Bangkok Smart Lighting (สำนักการโยธา กทม.)",พบไฟฟ้าส่องสว่างทางดับชำรุด จำนวนหลายดวง บริเวณ ซอยพหลโยธิน 54/2 เขตสายไหม\r\n#1555 #citydata,ซอย พหลโยธิน 54/2 แขวงสายไหม เขตสายไหม กรุงเทพมหานคร : WJHJ+9JF ซอย พหลโยธิน 54/2 แขวงสายไหม เขตสายไหม กรุงเทพมหานคร 10220 ประเทศไทย,สายไหม,สายไหม,2024,10,24


In [18]:
df.columns.tolist()

['type',
 'organization',
 'comment',
 'address',
 'subdistrict',
 'district',
 'year',
 'month',
 'day']

In [19]:
df

,type,organization,comment,address,subdistrict,district,year,month,day
ticket_id,,,,,,,,,
2021-FYJTFP,[ความสะอาด],เขตบางซื่อ,ขยะเยอะ,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,บางซื่อ,บางซื่อ,2021,9,3
2021-CGPMUN,"[น้ำท่วม, ร้องเรียน]","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆปีจะมีเครื่องสูบน้ำแต่ปีนี้ไม่มีกทม.ปล่อยทิ้ง ชุมชนเคหะนคร1แปลง2(ซ.เฉลิมพระเกียรติร.9ซอง22 วัดตะกล่ำ ประเวศ),189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเวศ กรุงเทพมหานคร 10250 ประเทศไทย,หนองบอน,ประเวศ,2021,9,19
2021-7XATFA,[สะพาน],เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,ยานนาวา,สาทร,2021,9,26
2021-9U2NJT,[น้ำท่วม],"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,บางซื่อ,บางซื่อ,2021,10,14
2021-DVEWYM,"[น้ำท่วม, ถนน]","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว\n\n1. หลังจากฝนตกต่อเนื่องประมาณ 30 นาที น้ำท่วมซอยสูงประมาณ 20 ซม ใช้เวลาระบายราวๆ 3-5 ชม. กว่าน้ำในซอยแห้ง\nหากฝนตกนานกว่า 1 ชม ใช้เวลา 2-3 วัน กว่าน้ำในซอยแห้ง\nหากนานกว่านั้น ต้นซอยน้ำแห้ง ปลายซอยน้ำเน่า,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว กรุงเทพมหานคร 10230 ประเทศไทย,ลาดพร้าว,ลาดพร้าว,2021,12,9
...,...,...,...,...,...,...,...,...,...
KDDNBP,[ร้องเรียน],เขตหนองจอก,ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบบการจัดการเรื่องราวร้องทุกข์ สำนักงานปลัดสำนักนายกรัฐมนตรี ทำเนียบรัฐบาล รหัส นร02680001844\n\nขอให้ตรวจสอบการจำหน่ายเครื่องดื่มแอลกอฮอล์ก่อนเวลาที่กฎหมายกำหนด บริเวณ ร้านจำหน่ายสินค้าเบ็ดเตล็ด ชื่อ .... (ไม่ทราบเลขที่ตั้ง) การเคหะหนองจอก ซอย 11ก แขวงคู้ฝั่งเหนือ เขตหนองจอก กรุงเทพมหานคร จุดสังเกต ร้านจะอยู่ด้านหน้าการเคหะหนองจอก ปากซอย 11ก ลักษณะเป็นร้านปูนสีขาว ชั้นเดียว เนื่องจากผู้ร้องแจ้งว่า ร้านดังกล่าวมีเจ้าของร้านเป็นบุคคลเพศชาย (ทราบชื่...,เขตหนองจอก กรุงเทพมหานคร : VV47+G6M แขวงกระทุ่มราย เขตหนองจอก กรุงเทพมหานคร 10530 ประเทศไทย,กระทุ่มราย,หนองจอก,2025,1,16
RYM99G,"[ความสะอาด, ถนน]","เขตคลองสาน,ฝ่ายสิ่งแวดล้อมฯ เขตคลองสาน",ปัญหา: บริเวณตลาดพระเครื่องพญาไม้ พบร้านอาหารตามสั่งและร้านก๋วยเตี๋ยว ประกอบอาหารส่งกลิ่นเหม็นและมีควันรบกวน ตั้งแต่เวลา 08.00-12.00 น.เป็นประจำทุกวัน ยกเว้นวันอาทิตย์ สร้างความเดือดร้อนให้กับผู้พักอาศัยบริเวณใกล้เคียง\nถนน: พญาไม้\nจุดสังเกต: อยู่บริเวณกลางตลาด\nเขต: คลองสาน\r\n#1555,39 ถ. สมเด็จเจ้าพระยา แขวงสมเด็จเจ้าพระยา เขตคลองสาน กรุงเทพมหานคร 10600 ประเทศไทย,สมเด็จเจ้าพระยา,คลองสาน,2025,1,16
2025-BDQ87Y,"[ร้องเรียน, ต้นไม้, ความปลอดภัย]",เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือนต้นไม้มันตายแล้วแห้งเหมือนจะล้มลงมาเกรงว่าจะเกิดอันตรายต่อผู้สัญจรอยากให้ช่วยมาตรวจสอบแก้ไขจุดสังเกตบริเวณใกล้ปากซอยราชวิถีซอย 7,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพมหานคร 10400 ประเทศไทย,ถนนพญาไท,ราชเทวี,2025,1,16


In [20]:
df[df.type.isna()].shape[0]

97

In [21]:
df = df[df['type'].notna()]

In [22]:
# Convert the column to string format momentarily to check unique values
df['type'].astype(str).value_counts()

# Now print it

type
['']                                                  114965
['ถนน']                                                85461
['ทางเท้า']                                            54431
['แสงสว่าง']                                           34122
['ความสะอาด']                                          32182
                                                       ...  
['ท่อระบายน้ำ', 'ถนน', 'เสนอแนะ', 'ทางเท้า']               1
['เสียงรบกวน', 'สะพาน', 'ความปลอดภัย', 'ทางเท้า']          1
['กีดขวาง', 'ท่อระบายน้ำ', 'น้ำท่วม', 'ร้องเรียน']         1
['สะพาน', 'ถนน', 'เสียงรบกวน', 'คลอง', 'ป้าย']             1
['กีดขวาง', 'ถนน', 'ทางเท้า', 'เสียงรบกวน']                1
Name: count, Length: 17443, dtype: int64

In [23]:
# 1. Explode the list column to separate the tags
all_individual_tags = df.explode('type')

# 2. Count the frequency of every single tag
# This gives you the "Master List" of categories you can use
tag_counts = all_individual_tags['type'].value_counts()

print("--- Top 20 Tags Found in Data ---")
print(tag_counts)

# Get the list of names for copy-pasting
unique_tags_list = tag_counts.index.tolist()


--- Top 20 Tags Found in Data ---
type
ถนน            255725
               114965
ทางเท้า        108748
ความปลอดภัย     81525
แสงสว่าง        72847
ความสะอาด       67451
กีดขวาง         66904
ร้องเรียน       50784
จราจร           46909
ท่อระบายน้ำ     42670
น้ำท่วม         41744
ป้าย            39079
เสียงรบกวน      35346
สะพาน           34403
ต้นไม้          29592
คลอง            25796
สายไฟ           24783
สัตว์จรจัด      16505
คนจรจัด          7414
PM2.5            7385
สอบถาม           6722
เสนอแนะ          5322
การเดินทาง       2337
ห้องน้ำ          1894
ป้ายจราจร        1422
Name: count, dtype: int64


In [24]:


# 1. Define the Keyword Map
# Dictionary Format: { 'Tag Name': [List of keywords that imply this tag] }
keyword_map = {
    'ถนน': ['หลุม', 'บ่อ', 'ยางมะตอย', 'ทรุด', 'พัง', 'ผิวจราจร', 'road', 'surface'],
    'ทางเท้า': ['ฟุตบาท', 'ทางเดิน', 'กระเบื้อง', 'sidewalk', 'footpath', 'อิฐ'],
    'แสงสว่าง': ['ไฟ', 'มืด', 'ดับ', 'ส่องสว่าง', 'light', 'lamp'],
    'ความสะอาด': ['ขยะ', 'สกปรก', 'เหม็น', 'รก', 'dirty', 'trash', 'rubbish'],
    'กีดขวาง': ['จอด', 'ขวาง', 'ร้านค้า', 'แผงลอย', 'block'],
    'ท่อระบายน้ำ': ['ท่อ', 'ฝา', 'ระบาย', 'drain'],
    'น้ำท่วม': ['ท่วม', 'ขัง', 'flood', 'water'],
    'จราจร': ['รถติด', 'รถมาก', 'traffic', 'jam'],
    'สายไฟ': ['สาย', 'ห้อย', 'ระโยง', 'cable', 'wire'],
    'ต้นไม้': ['ต้นไม้', 'กิ่ง', 'ตัด', 'หญ้า', 'tree', 'branch'],
    'สัตว์จรจัด': ['หมา', 'แมว', 'สุนัข', 'กัด', 'เห่า', 'dog', 'cat'],
    'เสียงรบกวน': ['เสียง', 'ดัง', 'เพลง', 'noise', 'loud'],
    'คลอง': ['คลอง', 'น้ำเสีย', 'canal'],
    'PM2.5': ['ฝุ่น', 'ควัน', 'dust', 'pm', 'อากาศ'],
    'ความปลอดภัย': ['อันตราย', 'โจร', 'ขโมย', 'เปลี่ยว', 'safe', 'crime'],
    'ป้าย': ['ป้าย', 'sign'],
    'สะพาน': ['สะพาน', 'bridge', 'overpass']
}

# 2. Define the Rescue Function
def recover_type_smart(row):
    # Get current values
    current_type = row['type']
    comment = str(row['comment']) # Convert to string to avoid errors

    # CHECK: Is the type currently empty?
    # We check for: empty list [], list with empty string [''], or None
    is_empty = False
    if isinstance(current_type, list):
        if len(current_type) == 0: is_empty = True
        elif len(current_type) == 1 and current_type[0] == '': is_empty = True
    elif pd.isna(current_type):
        is_empty = True

    # IF VALID: Return it as is (Don't change existing data)
    if not is_empty:
        return current_type

    # IF EMPTY: Try to guess based on keywords
    found_tags = set() # Use a set to avoid duplicates

    for tag, keywords in keyword_map.items():
        for word in keywords:
            if word in comment:
                found_tags.add(tag)
                # Once we match a keyword for this tag, stop checking other keywords for the same tag
                break

    # Return result
    if len(found_tags) > 0:
        return list(found_tags) # Convert back to list
    else:
        return ['Unspecified'] # Final fallback if no keywords match

# 3. Apply the function
# We create a NEW column first to be safe
df['type'] = df.apply(recover_type_smart, axis=1)

# 4. Check the Improvement
# Count how many are still 'Unspecified' vs how many we recovered
print("--- After Recovery ---")
# Explode again to count the new tags
print(df['type'].explode().value_counts().head(20))


--- After Recovery ---
type
ถนน            261793
ทางเท้า        111121
แสงสว่าง        85610
ความปลอดภัย     82964
กีดขวาง         80266
ความสะอาด       76958
Unspecified     54485
ร้องเรียน       50784
ท่อระบายน้ำ     48136
จราจร           47228
ป้าย            46350
น้ำท่วม         42950
เสียงรบกวน      40052
สะพาน           35539
ต้นไม้          33126
สายไฟ           28635
คลอง            27873
สัตว์จรจัด      22352
PM2.5           12009
คนจรจัด          7414
Name: count, dtype: int64


/var/folders/_8/dtm1t7kj7p77dph5_58cw6km0000gn/T/ipykernel_8883/2843960521.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'] = df.apply(recover_type_smart, axis=1)


In [25]:
target_tags = {'สัตว์จรจัด', 'ถนน', 'คนจรจัด'} # Use {} for a set

# Filter rows where the set of tags matches exactly
# (We use isinstance(x, list) to prevent errors if there are NaNs)
df[df['type'].apply(lambda x: set(x) == {'สัตว์จรจัด', 'ถนน', 'คนจรจัด'} if isinstance(x, list) else False)]


,type,organization,comment,address,subdistrict,district,year,month,day
ticket_id,,,,,,,,,
PTVRDY,"[ถนน, สัตว์จรจัด, คนจรจัด]","เขตธนบุรี,ฝ่ายสิ่งแวดล้อมฯ เขตธนบุรี",* ปัญหา: สุนัขจรจัดเข้ามาไล่กัดคน\n* จุดเกิดเหตุ: ภายในซอยวัดกันตทาราราม\n* บ้านเลขที่:-\n* ซอย: ซอยวัดกันตทาราราม\n* ถนน: ถนนราชพฤกษ์\n* เขต: เขตธนบุรี\n\nประชาชนแจ้งว่า พบสุนัขจรจัดเข้ามาไล่กัดคนบริเวณภายในซอยดังกล่าวคาดว่าน่าจะเป็นสุนัขของคนเร่ร่อนภายในซอย\r\n#1555,86/21 ซอย เทอดไท 19 แขวงตลาดพลู เขตธนบุรี กรุงเทพมหานคร 10600 ประเทศไทย,ตลาดพลู,ธนบุรี,2023,4,19
2023-EL9K9N,"[สัตว์จรจัด, ถนน, คนจรจัด]",เขตสวนหลวง,มีคนเร่ร่อน มาอาศัยนอนตรงข้างถนน มีสุนัขไว้เฝ้าของ สร้างกลิ่นเหม็น แถมถ้าเดินไปใกล้ๆของ ตอนเช้ามืด ต้องไปทำงาน มาเห่าจะกัดคนเดินเท้าไปมา ฝากดูแล ด้วยครีบ เป็น 3-4 ปีแล้วครับ,486 ถ. อ่อนนุช แขวงสวนหลวง กรุงเทพมหานคร 10250 ประเทศไทย,สวนหลวง,สวนหลวง,2023,8,2
876CMY,"[สัตว์จรจัด, ถนน, คนจรจัด]","เขตป้อมปราบศัตรูพ่าย,ฝ่ายเทศกิจ เขตป้อมปราบศัตรูพ่าย",ปัญหา: ประชาชนแจ้งว่า มีคนเร่ร่อนมานอน และให้อาหารสัตว์ ทั้งให้อาหารแมว สุนัข รวมถึงนก โดยเฉพาะนกเยอะมากกลัวนำเชื้อโรคเข้ามา และสกปรกมาก \nประชาชนต้องการให้เจ้าหน้าที่ตรวจสอบ\nซอย: เฉลิมเขต 1-2\nถนน: บำรุงเมือง\nจุดสังเกต: บริเวณอาคารพานิชย์ที่ร้างไม่มีคนอยู่ ในวงเวียนสวนมะลิ ถนนบำรุงเมือง เขตป้อมศัตรูพ่าย\nเขต: ป้อมปราบศัตรูพ่าย\r\n#1555,283 ซอย สวนมะลิ 1 แขวงวัดเทพศิรินทร์ เขตป้อมปราบศัตรูพ่าย กรุงเทพมหานคร 10100 ประเทศไทย,วัดเทพศิรินทร์,ป้อมปราบศัตรูพ่าย,2023,8,2
DTXU24,"[ถนน, คนจรจัด, สัตว์จรจัด]","เขตบางรัก,ฝ่ายพัฒนาชุมชนฯ เขตบางรัก",ปัญหา: พบมีคนเร่ร่อนมาอาศัยอยู่ริมถนน และเลี้ยงนกพิราบ ประชาชนเกรงว่าอาจจะเป็นแหล่งเแพร่เชื้อโรค \nถนน: มหาพฤฒาราม\nจุดสังเกต: บริเวณริมถนนดังกล่าว\nเขต: บางรัก\r\n#1555,15 ถนน มหาพฤฒาราม แขวงมหาพฤฒาราม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,มหาพฤฒาราม,บางรัก,2023,8,3
2024-E8LRCG,"[สัตว์จรจัด, ถนน, คนจรจัด]","เขตปทุมวัน,สวนหลวง (Suanluang),สำนักงานจัดการทรัพย์สิน จุฬาลงกรณ์มหาวิทยาลัย","1. (13.7420260, 100.5228615)\nพิกัดถนนบรรทัดทอง ถัดจากร้านขนมไทยแม่เดือนมา 2 ตึกแถว หรือถัดจากร้านหนึ่งนมนัวมา 3 ตึกแถว\n\n2. พบคนนำสุนัขมาขอทาน จากที่สังเกตได้เงินจากนักท่องเที่ยวต่างชาติมาบริจาคจำนวนมาก\nสังเกตในวันที่ 16 เมษายน 2567 ตั้งแต่เวลา 21.50-22.23 เป็นเวลาเกือบครึ่งชั่วโมง มีคนมาบริจาคไม่ต่ำกว่า 10 คน ส่วนใหญ่ต่างชาติให้ 20 บาท บางคนให้ถึง 200\n\nจากรูปที่ส่งจะเห็นว่าผู้หญิงเสื้อแดง เป็นเจ้าของสุนัข",1483 ถนน บรรทัดทอง แขวงวังใหม่ เขตปทุมวัน กรุงเทพมหานคร 10330 ประเทศไทย,วังใหม่,ปทุมวัน,2024,4,16
FVJUY6,"[ถนน, สัตว์จรจัด, คนจรจัด]","เขตห้วยขวาง,การรถไฟแห่งประเทศไทย",ปัญหา: ริมถนนดังกล่าว พบคนเร่ร่อนมาปลูกเพลิงพักอาศัยบนพื้นที่สาธารณะและลุกล้ำเข้าไปในที่ดินของผู้ร้อง อีกทั้งยังมีการเลี้ยงสุนัขจำนวนหลายตัว ซึ่งสุนัขดังกล่าวมีนิสัยดุร้าย มักกัดผู้คนที่สัญจรผ่านไปมา\nถนน: กำแพงเพชร 7\nจุดสังเกต: เลยวัดอุทัยธารามไปประมาณ 400 เมตร จะอยู่ทางด้านซ้ายมือ ติดกับแคมป์คนงานของบริษัทสยามชินเทค\nเขต: ห้วยขวาง\r\n#1555,4020 ถ. เพชรบุรี แขวงบางกะปิ เขตห้วยขวาง กรุงเทพมหานคร 10310 ประเทศไทย,บางกะปิ,ห้วยขวาง,2024,5,14
B8VGP4,"[ถนน, สัตว์จรจัด, คนจรจัด]","เขตสายไหม,ฝ่ายสิ่งแวดล้อมฯ เขตสายไหม",ปัญหา: บนสะพานลอยคนข้ามสะพานใหม่ (ตลาดยิ่งเจริญ) พบมีผู้นำสุนัข 2 ตัว มาผูกไว้โดยไม่มีการให้อาหารและน้ำ คาดว่ามาจากขอทานที่นำมาผูกไว้\nถนน: พหลโยธิน\nเขต: สายไหม\r\n#1555,128 ซ.แยกจากหมู่ 6 เดิม ถ. พหลโยธิน แขวงคลองถนน เขตสายไหม กรุงเทพมหานคร 10220 ประเทศไทย,คลองถนน,สายไหม,2024,9,14


In [26]:
df = df[~df['type'].apply(lambda x: set(x) == {'Unspecified'} if isinstance(x, list) else False)]
df.shape[0]


731048

In [27]:
df[['type', 'comment','organization']].sample(24)


,type,comment,organization
ticket_id,,,
GPMXUA,[ถนน],ปัญหา: บริเวณปากซอยดังกล่าว พบแท่งปูนที่สร้างติดกับถนน เพื่อกันรถเข้ามาจอด จึงต้องการให้ตรวจสอบว่าสร้างถูกต้องตามกฎหมายกำหนดหรือไม่\nซอย: ปุณณวิถี 6\nถนน: สุขุมวิท 101\nเขต: พระโขนง\r\n#1555,"เขตพระโขนง,ฝ่ายโยธา เขตพระโขนง"
PRN4YF,[น้ำท่วม],น้ำท่วมบ่อปลามีเจ้าหน้าที่ฝ่ายพัฒนาชุมชนและเจ้าหน้าที่เกษตรลงตรวจสอบแล้วน้ำท่วมบ่อปลาแต่ไม่มีบัตรประมงแต่มีบัตรเกษตรส่งเอกสารครบ สอบปากคำเรียบร้อยแล้ว เจ้าหน้าที่ฝ่ายพัฒนาชุมชนแจ้งว่าไม่เกินเดือนมกราคมเงินช่วยเหลือน้ำท่วมบ่อปลาจะเข้าบัญชี ณ วันที่ 25 กุมภาพันธ์ 2566 เงินยังไม่เข้าบัญชี แล้วจะนำเงินที่ไหนมาลงทุนเลี้ยงปลาต่อค่ะ มีการทุจริตไหมภายในองค์กรไหมค่ะ เอาเอกสารเกษตรไปครบทุกอย่าง ในบิลอาหารปลา บิลลูกปลา ตกลงคืออย่างไงค่ะ\r\n#1555 #bkkrongtook,"เขตลาดกระบัง,ฝ่ายพัฒนาชุมชนฯ เขตลาดกระบัง"
2022-AFDN2Z,[ถนน],1. ถนนซ่อมไม่เสร็จสักที เส้นบางโพ จนเกิดอุบัติเหตุรถล้ม,"เขตบางซื่อ,สำนักการโยธา กทม.,ฝ่ายโยธา เขตบางซื่อ,การไฟฟ้านครหลวง เขตสามเสน,การไฟฟ้านครหลวง MEA,ผอ.เขตบางซื่อ (นายเจษฎา),กลุ่มกรุงเทพเหนือ (นางวันทนีย์ วัฒนะ)"
2024-7GCTDE,[ถนน],Singapore Hawker หาบเร่ center model...ได้รับการขึ้นทะเบียนเป็นมรดกทางวัฒนธรรมที่จับต้องไม่ได้ UNESCO...ผอ เขตพระนครแจ้งว่าจะเอา งานถนนคนเดินมหรรณพเป็นโมเดลในการจัดงานปิดถนนคนเดินในชุมชนอื่นๆของเขตพระนครต่อไป ได้ยิน ผอ บอกแบบนี้แล้วรู้สึกตกใจเลย คิดไม่ถึงว่าจะได้ยินแบบนี้ ทีมเทศกิจประกาศแจ้งในงานว่าจะปิดถนนทุกอาทิตย์ให้ดังไปถึงโลกด้วย ...,"เขตพระนคร,ฝ่ายเทศกิจ เขตพระนคร,ฝ่ายพัฒนาชุมชนฯ เขตพระนคร"
W6298B,"[ความปลอดภัย, ถนน]",ปัญหา: บริเวณปากซอยดังกล่าว ดังกล่าวพบ ผิวการจราจรเป็นหลุมเป็นบ่อ ส่งผลให้หากมีรถจักรยานยนต์ที่ขับขี่ด้วยความเร็วจะเกิดอุบัติเหตุได้\nซอย: รามคำแหง 42\nถนน: รามคำแหง\nจุดสังเกต: หน้าปากซอยดังกล่าวมุ่งหน้าสุขาภิบาล 3\nเขต: บางกะปิ\r\n#1555,"กองโครงการสายอากาศเป็นสายใต้ดิน2 ฝ่ายบริหารโครงการ การไฟฟ้านครหลวง (กคต.2 ฝบค. กฟน.),โครงการตามแนวรถไฟฟ้าสายสีส้ม สัญญา 3,โครงการ DB สายสีส้ม สญ.3 ฝอบ. กฟน.,การไฟฟ้านครหลวง เขตบางกะปิ,การไฟฟ้านครหลวง ฝ่ายบริหารโครงการ,เขตบางกะปิ,กองแผนงานและประสานสาธารณูปโภค สำนักการโยธา (กผภ.สนย.) กทม.,การไฟฟ้านครหลวง MEA,สำนักการโยธา กทม.,ฝ่ายโยธา เขตบางกะปิ"
2024-PDZNNB,[ป้าย],ป้าย,"เขตหนองแขม,ฝ่ายเทศกิจ เขตหนองแขม"
J9VTKT,[เสียงรบกวน],ภายในซอยพัฒนาการ 20 แยก 4 เข้ามาสุดซอยเจอบริษัทน้ำอัดลม แล้วเลี้ยวซ้ายไปสุดทาง มีคนหรือหลุ่มคนชอบเปิดเครื่องเสียงที่รถกระบะสีน้ำเงินเสียงดังเป็นเวลานาน\nในตอนแรกคิดว่าคงเปิดแค่วันหรือสองวันคงเพราะอาจมีงานสังสรรค์ แต่ปัจจุบัน(22-12-2024)\nก็มันจะเปิดเพลงเสียงดัง เสียงทุ้ม (เสียง Bass) ดังรบกวนเป็นบริเวณกว้างมาหลายวันแล้ว\r\n#1555 #bkkrongtook,"เขตสวนหลวง,สน.คลองตัน,สำนักงานตำรวจแห่งชาติ (Royal Thai Police),กองบัญชาการตำรวจนครบาล (บช.น.),กองบังคับการตำรวจนครบาล 5 (บก.น.5)"
ARYHT3,[แสงสว่าง],ปัญหา : สาธารณูปโภค เนื่องจาก ไฟดับ 1 ดวง ดับ 1 เดือนแล้ว .ขอให้ดำเนินการตรวจสอบและแก้ไขปัญหานี้ด้วย\nพิกัดจุดสังเกต :จะอยู่โค้งขวามือ เสาร์ไฟฟ้าเลขที่ ข3\nซอย : รามอินทรา 34 แยก 12\nเขต :บางเขน \nแขวง:ท่าแร้ง\r\n#1555,"เขตบางเขน,การไฟฟ้านครหลวง เขตบางเขน,ฝ่ายโยธา เขตบางเขน,การไฟฟ้านครหลวง MEA"
A9E64H,"[ถนน, ร้องเรียน, เสียงรบกวน]",ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบบการจัดการเรื่องราวร้องทุกข์ สำนักงานปลัดสำนักนายกรัฐมนตรี ทำเนียบรัฐบาล รหัสเรื่อง นร02660012404\n\nขอให้แก้ไขปัญหาเสียงดังรบกวน และฝุ่นละอองฟุ้งกระจาย จากการก่อสร้างอาคาร บริเวณ คอนโดมิเนียม ชื่อ ดีคอนโด พนา เลขที่ตั้ง 188 ถนนเลียบทางรถไฟตลิ่งชัน แขวงบางขุนศรี เขตบางกอกน้อย กรุงเทพมหานคร เนื่องจากผู้ร้องแจ้งว่า ในพื้นที่ดังกล่าวมีการดำเนินการก่อสร้างอาคารคอนโดมิเนียม โดยจะมีเจ้าหน้าที่วิศวกร ฝ่ายโยธา (ทราบชื่อเล่น) (ไม่ระบุรูปพรรณสัณฐาน) เป็นผู...,เขตบางกอกน้อย


In [28]:
# drop if index is empty
df = df[~df.index.isna()]
int(df.index.isna().sum())


0

before split

In [29]:
df.shape[0]

724915

## Split list values into multiple rows

In [30]:
df = df.explode('type')
df


,type,organization,comment,address,subdistrict,district,year,month,day
ticket_id,,,,,,,,,
2021-FYJTFP,ความสะอาด,เขตบางซื่อ,ขยะเยอะ,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,บางซื่อ,บางซื่อ,2021,9,3
2021-CGPMUN,น้ำท่วม,"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆปีจะมีเครื่องสูบน้ำแต่ปีนี้ไม่มีกทม.ปล่อยทิ้ง ชุมชนเคหะนคร1แปลง2(ซ.เฉลิมพระเกียรติร.9ซอง22 วัดตะกล่ำ ประเวศ),189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเวศ กรุงเทพมหานคร 10250 ประเทศไทย,หนองบอน,ประเวศ,2021,9,19
2021-CGPMUN,ร้องเรียน,"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆปีจะมีเครื่องสูบน้ำแต่ปีนี้ไม่มีกทม.ปล่อยทิ้ง ชุมชนเคหะนคร1แปลง2(ซ.เฉลิมพระเกียรติร.9ซอง22 วัดตะกล่ำ ประเวศ),189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเวศ กรุงเทพมหานคร 10250 ประเทศไทย,หนองบอน,ประเวศ,2021,9,19
2021-7XATFA,สะพาน,เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,ยานนาวา,สาทร,2021,9,26
2021-9U2NJT,น้ำท่วม,"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,บางซื่อ,บางซื่อ,2021,10,14
...,...,...,...,...,...,...,...,...,...
2025-BDQ87Y,ร้องเรียน,เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือนต้นไม้มันตายแล้วแห้งเหมือนจะล้มลงมาเกรงว่าจะเกิดอันตรายต่อผู้สัญจรอยากให้ช่วยมาตรวจสอบแก้ไขจุดสังเกตบริเวณใกล้ปากซอยราชวิถีซอย 7,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพมหานคร 10400 ประเทศไทย,ถนนพญาไท,ราชเทวี,2025,1,16
2025-BDQ87Y,ต้นไม้,เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือนต้นไม้มันตายแล้วแห้งเหมือนจะล้มลงมาเกรงว่าจะเกิดอันตรายต่อผู้สัญจรอยากให้ช่วยมาตรวจสอบแก้ไขจุดสังเกตบริเวณใกล้ปากซอยราชวิถีซอย 7,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพมหานคร 10400 ประเทศไทย,ถนนพญาไท,ราชเทวี,2025,1,16
2025-BDQ87Y,ความปลอดภัย,เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือนต้นไม้มันตายแล้วแห้งเหมือนจะล้มลงมาเกรงว่าจะเกิดอันตรายต่อผู้สัญจรอยากให้ช่วยมาตรวจสอบแก้ไขจุดสังเกตบริเวณใกล้ปากซอยราชวิถีซอย 7,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพมหานคร 10400 ประเทศไทย,ถนนพญาไท,ราชเทวี,2025,1,16


In [31]:
print(df.shape)
df.to_csv('prep1.csv', index=False)

(1148903, 9)


In [32]:
# Drop the 'comment' column
df.drop(columns=['comment','address','organization','district'], inplace=True)

# Check the new shape of the DataFrame
df.shape
df

,type,subdistrict,year,month,day
ticket_id,,,,,
2021-FYJTFP,ความสะอาด,บางซื่อ,2021,9,3
2021-CGPMUN,น้ำท่วม,หนองบอน,2021,9,19
2021-CGPMUN,ร้องเรียน,หนองบอน,2021,9,19
2021-7XATFA,สะพาน,ยานนาวา,2021,9,26
2021-9U2NJT,น้ำท่วม,บางซื่อ,2021,10,14
...,...,...,...,...,...
2025-BDQ87Y,ร้องเรียน,ถนนพญาไท,2025,1,16
2025-BDQ87Y,ต้นไม้,ถนนพญาไท,2025,1,16
2025-BDQ87Y,ความปลอดภัย,ถนนพญาไท,2025,1,16


In [33]:
df['date_obj'] = pd.to_datetime(df[['year', 'month', 'day']])

In [34]:
daily_data = pd.crosstab(
    index=[df['date_obj'], df['subdistrict']],
    columns=df['type']
).reset_index()

In [35]:
target_cols = [c for c in daily_data.columns if c not in ['date_obj', 'subdistrict']]

In [36]:
daily_data['year'] = daily_data['date_obj'].dt.year
daily_data['month'] = daily_data['date_obj'].dt.month
daily_data['day'] = daily_data['date_obj'].dt.day

In [37]:
# -------------------------------
# Convert date and sort
# -------------------------------
daily_data['date_obj'] = pd.to_datetime(daily_data['date_obj'])
daily_data = daily_data.sort_values(by='date_obj').reset_index(drop=True)

# -------------------------------
# Feature engineering (time features)
# -------------------------------
daily_data['month'] = daily_data['date_obj'].dt.month
daily_data['day_of_week'] = daily_data['date_obj'].dt.dayofweek
daily_data['day_of_year'] = daily_data['date_obj'].dt.dayofyear

daily_data['month_sin'] = np.sin(2 * np.pi * daily_data['month'] / 12)
daily_data['month_cos'] = np.cos(2 * np.pi * daily_data['month'] / 12)

daily_data['dow_sin'] = np.sin(2 * np.pi * daily_data['day_of_week'] / 7)
daily_data['dow_cos'] = np.cos(2 * np.pi * daily_data['day_of_week'] / 7)

daily_data['doy_sin'] = np.sin(2 * np.pi * daily_data['day_of_year'] / 365)
daily_data['doy_cos'] = np.cos(2 * np.pi * daily_data['day_of_year'] / 365)

# -------------------------------
# Target columns
# -------------------------------
target_cols = [
    'ถนน', 'ทางเท้า', 'แสงสว่าง', 'ความสะอาด', 'กีดขวาง',
    'ท่อระบายน้ำ', 'น้ำท่วม', 'จราจร', 'สายไฟ', 'ต้นไม้',
    'สัตว์จรจัด', 'เสียงรบกวน', 'คลอง', 'PM2.5', 'ความปลอดภัย',
    'ป้าย', 'สะพาน','การเดินทาง','คนจรจัด','ป้ายจราจร','ร้องเรียน','สอบถาม','ห้องน้ำ','เสนอแนะ'
]

cols_to_drop = target_cols + ['date_obj', 'month', 'day_of_week', 'day_of_year']

# -------------------------------
# Split ranges
# -------------------------------
train_start_date = pd.Timestamp('2023-01-01')
train_end_date   = pd.Timestamp('2024-06-30')

val_start_date   = pd.Timestamp('2024-07-01')
val_end_date     = pd.Timestamp('2024-09-30')

test_start_date  = pd.Timestamp('2024-10-01')
test_end_date    = pd.Timestamp('2025-01-31')

# -------------------------------
# Split datasets
# -------------------------------
train_df = daily_data[(daily_data['date_obj'] >= train_start_date) &
                      (daily_data['date_obj'] <= train_end_date)].copy()

val_df   = daily_data[(daily_data['date_obj'] >= val_start_date) &
                      (daily_data['date_obj'] <= val_end_date)].copy()

test_df  = daily_data[(daily_data['date_obj'] >= test_start_date) &
                      (daily_data['date_obj'] <= test_end_date)].copy()

# -------------------------------
# Prepare X, y
# -------------------------------
X_train = train_df.drop(columns=cols_to_drop)
y_train = train_df[target_cols]

X_val = val_df.drop(columns=cols_to_drop)
y_val = val_df[target_cols]

X_test = test_df.drop(columns=cols_to_drop)
y_test = test_df[target_cols]

# -------------------------------
# Print summary
# -------------------------------
print("Train range:", train_df['date_obj'].min(), "→", train_df['date_obj'].max())
print("Val range:",   val_df['date_obj'].min(),   "→", val_df['date_obj'].max())
print("Test range:",  test_df['date_obj'].min(),  "→", test_df['date_obj'].max())

print(f"Train Shape: {X_train.shape}, Val Shape: {X_val.shape}, Test Shape: {X_test.shape}")


Train range: 2023-01-01 00:00:00 → 2024-06-30 00:00:00
Val range: 2024-07-01 00:00:00 → 2024-09-30 00:00:00
Test range: 2024-10-01 00:00:00 → 2025-01-16 00:00:00
Train Shape: (80005, 9), Val Shape: (13883, 9), Test Shape: (16086, 9)


In [ ]:
X_train

type,ถนน,ทางเท้า,แสงสว่าง,ความสะอาด,กีดขวาง,ท่อระบายน้ำ,น้ำท่วม,จราจร,สายไฟ,ต้นไม้,...,ความปลอดภัย,ป้าย,สะพาน,การเดินทาง,คนจรจัด,ป้ายจราจร,ร้องเรียน,สอบถาม,ห้องน้ำ,เสนอแนะ
28845,1,2,0,1,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
28846,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28847,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28848,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28849,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108845,2,2,0,0,0,1,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
108846,0,0,0,0,1,0,0,0,0,2,...,3,0,0,0,0,0,0,0,0,0
108847,1,0,0,0,0,0,1,1,1,0,...,1,0,2,0,0,0,0,0,0,0
108848,1,4,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
model = CatBoostRegressor(
    loss_function='MultiRMSE',
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    border_count=128,
    random_seed=42,
    task_type='CPU',
    thread_count=-1,
    verbose=100
)

model.fit(
    X_train, y_train,
    cat_features=['subdistrict'],
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=200
)


0:	learn: 4.6051320	test: 4.8448047	best: 4.8448047 (0)	total: 422ms	remaining: 14m 4s
100:	learn: 4.1751073	test: 4.4362316	best: 4.4362316 (100)	total: 51.5s	remaining: 16m 9s
200:	learn: 4.1130713	test: 4.4210701	best: 4.4210701 (200)	total: 1m 51s	remaining: 16m 38s
300:	learn: 4.0683566	test: 4.4304310	best: 4.4207590 (217)	total: 2m 54s	remaining: 16m 24s
400:	learn: 4.0169869	test: 4.4419971	best: 4.4207590 (217)	total: 4m	remaining: 15m 57s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 4.42075904
bestIteration = 217

Shrink model to first 218 iterations.


In [40]:
from sklearn.metrics import mean_absolute_error

# 1. ทำนายค่า (Predict) จากชุดข้อมูล Validation
y_pred = model.predict(X_val)

# 2. คำนวณค่า MAE ภาพรวม (Overall)
mae_overall = mean_absolute_error(y_val, y_pred)
print(f"Overall MAE: {mae_overall:.4f}")

print("-" * 30)

# 3. คำนวณค่า MAE แยกรายคอลัมน์ (เนื่องจากโมเดลทำนายหลายตัวแปรพร้อมกัน)
# ตรวจสอบว่า y_val เป็น DataFrame เพื่อดึงชื่อคอลัมน์มาแสดง
mae_per_col = mean_absolute_error(y_val, y_pred, multioutput='raw_values')

if hasattr(y_val, 'columns'):
    for i, col_name in enumerate(y_val.columns):
        print(f"MAE for {col_name}: {mae_per_col[i]:.4f}")
else:
    # กรณี y_val เป็น numpy array ไม่มีชื่อคอลัมน์
    for i, val in enumerate(mae_per_col):
        print(f"MAE for Target {i}: {val:.4f}")

Overall MAE: 0.4258
------------------------------
MAE for ถนน: 1.3079
MAE for ทางเท้า: 0.8676
MAE for แสงสว่าง: 0.6479
MAE for ความสะอาด: 0.6508
MAE for กีดขวาง: 0.7381
MAE for ท่อระบายน้ำ: 0.4803
MAE for น้ำท่วม: 0.4857
MAE for จราจร: 0.5032
MAE for สายไฟ: 0.3078
MAE for ต้นไม้: 0.4452
MAE for สัตว์จรจัด: 0.2937
MAE for เสียงรบกวน: 0.3984
MAE for คลอง: 0.3006
MAE for PM2.5: 0.1378
MAE for ความปลอดภัย: 0.7254
MAE for ป้าย: 0.5588
MAE for สะพาน: 0.3643
MAE for การเดินทาง: 0.0408
MAE for คนจรจัด: 0.1344
MAE for ป้ายจราจร: 0.0299
MAE for ร้องเรียน: 0.5622
MAE for สอบถาม: 0.1162
MAE for ห้องน้ำ: 0.0367
MAE for เสนอแนะ: 0.0856


In [41]:
import pickle

# Save the trained model
model.save_model('bangkok_traffy_model.cbm')

# Save feature names and target columns for API
model_metadata = {
    'feature_names': X_train.columns.tolist(),
    'target_cols': target_cols,
    'subdistricts': X_train['subdistrict'].unique().tolist()
}

with open('model_metadata.pkl', 'wb') as f:
    pickle.dump(model_metadata, f)

print("✓ Model saved as 'bangkok_traffy_model.cbm'")
print("✓ Metadata saved as 'model_metadata.pkl'")
print(f"✓ Features: {len(model_metadata['feature_names'])}")
print(f"✓ Target categories: {len(target_cols)}")
print(f"✓ Subdistricts: {len(model_metadata['subdistricts'])}")

✓ Model saved as 'bangkok_traffy_model.cbm'
✓ Metadata saved as 'model_metadata.pkl'
✓ Features: 9
✓ Target categories: 24
✓ Subdistricts: 177


## Visualization (Viz)

In [42]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from folium import plugins

### Graph Visualization 1: Top 10 Subdistricts by Total Complaints

In [43]:
# Calculate total complaints by subdistrict
subdistrict_totals = daily_data[target_cols].sum(axis=1).groupby(daily_data['subdistrict']).sum()
top_10_subdistricts = subdistrict_totals.nlargest(10).sort_values()

# Create horizontal bar chart
fig = go.Figure(go.Bar(
    x=top_10_subdistricts.values,
    y=top_10_subdistricts.index,
    orientation='h',
    marker=dict(
        color=top_10_subdistricts.values,
        colorscale='Reds',
        showscale=True,
        colorbar=dict(title="Complaints")
    ),
    text=top_10_subdistricts.values,
    textposition='auto',
))

fig.update_layout(
    title='Top 10 Subdistricts by Total Complaints',
    xaxis_title='Total Complaints',
    yaxis_title='Subdistrict',
    height=500,
    template='plotly_white'
)

fig.show()

### Graph Visualization 2: Complaint Types Distribution (Sunburst Chart)

In [44]:
# Calculate total complaints by type
complaint_type_totals = daily_data[target_cols].sum().sort_values(ascending=False)

# Create sunburst chart
fig = go.Figure(go.Sunburst(
    labels=['Total'] + complaint_type_totals.index.tolist(),
    parents=[''] + ['Total'] * len(complaint_type_totals),
    values=[complaint_type_totals.sum()] + complaint_type_totals.tolist(),
    marker=dict(colorscale='RdYlBu_r'),
    textinfo='label+percent parent'
))

fig.update_layout(
    title='Complaint Types Distribution (Interactive Sunburst)',
    height=600,
    template='plotly_white'
)

fig.show()

### Graph Visualization 3: Time Series - Complaints Over Time

In [45]:
# Aggregate complaints by date
daily_totals = daily_data.groupby('date_obj')[target_cols].sum()
daily_totals['total'] = daily_totals.sum(axis=1)

# Create time series plot
fig = go.Figure()

# Add total complaints line
fig.add_trace(go.Scatter(
    x=daily_totals.index,
    y=daily_totals['total'],
    mode='lines',
    name='Total Complaints',
    line=dict(color='#e74c3c', width=2)
))

# Add top 5 complaint types
top_5_types = complaint_type_totals.head(5).index
colors = ['#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c']

for i, complaint_type in enumerate(top_5_types):
    fig.add_trace(go.Scatter(
        x=daily_totals.index,
        y=daily_totals[complaint_type],
        mode='lines',
        name=complaint_type,
        line=dict(color=colors[i], width=1.5),
        visible='legendonly'
    ))

fig.update_layout(
    title='Complaints Over Time (Daily)',
    xaxis_title='Date',
    yaxis_title='Number of Complaints',
    height=500,
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    )
)

fig.show()

### Graph Visualization 4: Heatmap - Complaint Types by Top Subdistricts

In [46]:
# Get top 15 subdistricts
top_15_subdistricts = subdistrict_totals.nlargest(15).index

# Create heatmap data
heatmap_data = daily_data[daily_data['subdistrict'].isin(top_15_subdistricts)].groupby('subdistrict')[target_cols].sum()

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='YlOrRd',
    text=heatmap_data.values,
    texttemplate='%{text:.0f}',
    textfont={"size": 8},
    colorbar=dict(title="Complaints")
))

fig.update_layout(
    title='Heatmap: Complaint Types by Top 15 Subdistricts',
    xaxis_title='Complaint Type',
    yaxis_title='Subdistrict',
    height=600,
    template='plotly_white'
)

fig.update_xaxes(tickangle=-45)
fig.show()

In [47]:
import plotly.graph_objects as go

# 1. กำหนดคำค้นหาเกี่ยวกับน้ำ (Water Keywords)
water_keywords = ['น้ำท่วม', 'คลอง', 'ท่อระบายน้ำ', 'ระบายน้ำ']

# คัดเลือกคอลัมน์ที่มีคำค้นหาเหล่านี้อยู่
water_cols = [col for col in daily_data.columns if any(kw in col for kw in water_keywords)]

print(f"หัวข้อปัญหาที่เกี่ยวกับน้ำที่พบ: {water_cols}")

# 2. เตรียมข้อมูล (Data Preparation)
# ตรวจสอบว่าคอลัมน์ date เป็น datetime object
daily_data['date_obj'] = pd.to_datetime(daily_data['date_obj'])

# สร้าง DataFrame ชั่วคราวเพื่อไม่ให้กระทบข้อมูลหลัก
df_water = daily_data.copy()

# สร้างคอลัมน์เดือน (Format: YYYY-MM)
df_water['month_str'] = df_water['date_obj'].dt.strftime('%Y-%m')

# รวมยอดปัญหาเกี่ยวกับน้ำทั้งหมดในแต่ละแถว
df_water['total_water_issues'] = df_water[water_cols].sum(axis=1)

# Group by 'district' และ 'month_str', แล้ว sum ยอดรวม
water_grouped = df_water.groupby(['subdistrict', 'month_str'])['total_water_issues'].sum().reset_index()

# Pivot ตาราง: แถว=เขต, คอลัมน์=เดือน, ค่า=จำนวนเรื่องร้องเรียน
heatmap_matrix = water_grouped.pivot(index='subdistrict', columns='month_str', values='total_water_issues').fillna(0)

# (Option) คัดเลือกเฉพาะ Top 15 เขตที่มีปัญหาน้ำรวมสูงสุด เพื่อให้กราฟดูง่ายขึ้น (ถ้าต้องการดูทุกเขต ให้ comment 2 บรรทัดนี้ทิ้ง)
top_districts = heatmap_matrix.sum(axis=1).nlargest(15).index
heatmap_matrix = heatmap_matrix.loc[top_districts]

# 3. สร้าง Heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_matrix.values,
    x=heatmap_matrix.columns,
    y=heatmap_matrix.index,
    colorscale='Blues', # ใช้โทนสีฟ้าสื่อถึงน้ำ
    text=heatmap_matrix.values,
    texttemplate='%{text:.0f}', # แสดงตัวเลข
    textfont={"size": 10},
    colorbar=dict(title="จำนวนเรื่องร้องเรียน")
))

fig.update_layout(
    title='Heatmap: สถิตอปัญหาเกี่ยวกับน้ำ (น้ำท่วม/คลอง/ท่อ) รายเขต แยกรายเดือน',
    xaxis_title='เดือน',
    yaxis_title='เขต (District)',
    height=800, # เพิ่มความสูงเผื่อมีหลายเขต
    template='plotly_white'
)

fig.update_xaxes(tickangle=-45) # เอียงตัวหนังสือแกน X ให้อ่านง่าย
fig.show()

หัวข้อปัญหาที่เกี่ยวกับน้ำที่พบ: ['คลอง', 'ท่อระบายน้ำ', 'น้ำท่วม']


### Geospatial Visualization 1: Bangkok Subdistrict Map with Complaint Intensity

In [48]:
# Approximate coordinates for Bangkok subdistricts (top areas)
# Note: In production, you would use actual GeoJSON data or geocoding API
bangkok_coords = {
    'วังทองหลาง': (13.7783, 100.6028),
    'ลาดพร้าว': (13.8158, 100.6060),
    'สายไหม': (13.9017, 100.6536),
    'บางนา': (13.6686, 100.6252),
    'ดินแดง': (13.7636, 100.5560),
    'คลองตัน': (13.7297, 100.5805),
    'บางกะปิ': (13.7630, 100.6428),
    'สามเสนใน': (13.7969, 100.5362),
    'คลองเตย': (13.7214, 100.5809),
    'จตุจักร': (13.8116, 100.5532),
}

# Calculate total complaints per subdistrict
subdistrict_complaints = daily_data.groupby('subdistrict')[target_cols].sum().sum(axis=1).to_dict()

# Create base map centered on Bangkok
bangkok_map = folium.Map(
    location=[13.7563, 100.5018],
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Add circle markers for subdistricts
for subdistrict, coords in bangkok_coords.items():
    if subdistrict in subdistrict_complaints:
        complaints = subdistrict_complaints[subdistrict]

        # Size based on complaint count
        radius = (complaints / max(subdistrict_complaints.values())) * 30 + 5

        folium.CircleMarker(
            location=coords,
            radius=radius,
            popup=f'<b>{subdistrict}</b><br>Complaints: {complaints:.0f}',
            tooltip=f'{subdistrict}: {complaints:.0f} complaints',
            color='red',
            fill=True,
            fillColor='red',
            fillOpacity=0.6,
            weight=2
        ).add_to(bangkok_map)

# Add legend
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 220px; height: 120px;
     background-color: white; border:2px solid grey; z-index:9999;
     font-size:14px; padding: 10px">
     <p style="margin:0"><b>Bangkok Traffy Complaints</b></p>
     <p style="margin:5px 0">Circle size = Complaint volume</p>
     <p style="margin:5px 0; color:red">● High complaint areas</p>
     <p style="margin:5px 0; font-size:11px">Sample of major subdistricts</p>
</div>
'''
bangkok_map.get_root().html.add_child(folium.Element(legend_html))

# Display map
bangkok_map

### Geospatial Visualization 2: Heat Map of Complaint Density

In [49]:
# Create heatmap data - repeat coordinates by complaint intensity
heat_data = []
for subdistrict, coords in bangkok_coords.items():
    if subdistrict in subdistrict_complaints:
        complaints = subdistrict_complaints[subdistrict]
        # Normalize to create intensity (0-1 scale for better visualization)
        intensity = complaints / max(subdistrict_complaints.values())
        # Add points proportional to complaints (more complaints = more heat)
        num_points = int(intensity * 50) + 5
        for _ in range(num_points):
            # Add slight random offset for better heat distribution
            lat = coords[0] + np.random.normal(0, 0.01)
            lon = coords[1] + np.random.normal(0, 0.01)
            heat_data.append([lat, lon])

# Create heatmap
heat_map = folium.Map(
    location=[13.7563, 100.5018],
    zoom_start=11,
    tiles='CartoDB positron'
)

# Add heatmap layer
plugins.HeatMap(
    heat_data,
    radius=15,
    blur=20,
    max_zoom=13,
    gradient={0.4: 'blue', 0.65: 'lime', 0.8: 'yellow', 1.0: 'red'}
).add_to(heat_map)

# Add title
title_html = '''
<div style="position: fixed;
     top: 10px; left: 50px; width: 300px; height: 50px;
     background-color: white; border:2px solid grey; z-index:9999;
     font-size:16px; padding: 10px">
     <b>Bangkok Complaint Density Heatmap</b>
</div>
'''
heat_map.get_root().html.add_child(folium.Element(title_html))

heat_map

### Graph Visualization 5: Network Graph - Subdistrict Similarity by Complaint Patterns

In [50]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

# Get top 20 subdistricts for network graph
top_20_subdistricts = subdistrict_totals.nlargest(20).index

# Create complaint profile for each subdistrict
subdistrict_profiles = daily_data[daily_data['subdistrict'].isin(top_20_subdistricts)].groupby('subdistrict')[target_cols].sum()

# Calculate cosine similarity between subdistricts
similarity_matrix = cosine_similarity(subdistrict_profiles)

# Create network graph
G = nx.Graph()

# Add nodes
for subdistrict in top_20_subdistricts:
    G.add_node(subdistrict, size=subdistrict_totals[subdistrict])

# Add edges based on similarity (threshold > 0.85)
threshold = 0.85
for i, sub1 in enumerate(top_20_subdistricts):
    for j, sub2 in enumerate(top_20_subdistricts):
        if i < j and similarity_matrix[i, j] > threshold:
            G.add_edge(sub1, sub2, weight=similarity_matrix[i, j])

# Create layout
pos = nx.spring_layout(G, k=2, iterations=50, seed=42)

# Prepare edge traces
edge_x = []
edge_y = []
edge_weights = []

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_weights.append(edge[2]['weight'])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Prepare node traces
node_x = []
node_y = []
node_text = []
node_size = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    complaints = subdistrict_totals[node]
    node_text.append(f'{node}<br>Complaints: {complaints:.0f}<br>Connections: {G.degree(node)}')
    node_size.append(np.sqrt(complaints) / 5)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=[node for node in G.nodes()],
    textposition="top center",
    textfont=dict(size=8),
    hoverinfo='text',
    hovertext=node_text,
    marker=dict(
        showscale=True,
        colorscale='YlOrRd',
        size=node_size,
        color=[subdistrict_totals[node] for node in G.nodes()],
        colorbar=dict(
            thickness=15,
            title='Total Complaints',
            xanchor='left'
        ),
        line=dict(width=2, color='white')
    )
)

# Create figure
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title=dict(
                    text=f'Subdistrict Similarity Network (Top 20, Similarity > {threshold})',
                    font=dict(size=16)
                ),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20, l=5, r=5, t=40),
                annotations=[dict(
                    text="Node size & color = Complaint volume | Edges = Similar complaint patterns",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002
                )],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                height=700,
                template='plotly_white'
                )
            )

fig.show()

### Graph Visualization 6: 3D Scatter - Temporal and Spatial Distribution

In [51]:
# Sample data for 3D visualization (top 10 subdistricts, monthly aggregation)
sample_data = daily_data[daily_data['subdistrict'].isin(top_10_subdistricts.index)].copy()
sample_data['month_year'] = sample_data['date_obj'].dt.to_period('M').astype(str)
sample_data['total_complaints'] = sample_data[target_cols].sum(axis=1)

# Aggregate by month and subdistrict
monthly_agg = sample_data.groupby(['month_year', 'subdistrict'])['total_complaints'].sum().reset_index()

# Create month index for plotting
monthly_agg['month_idx'] = pd.factorize(monthly_agg['month_year'])[0]
monthly_agg['subdistrict_idx'] = pd.factorize(monthly_agg['subdistrict'])[0]

# Create 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=monthly_agg['month_idx'],
    y=monthly_agg['subdistrict_idx'],
    z=monthly_agg['total_complaints'],
    mode='markers',
    marker=dict(
        size=5,
        color=monthly_agg['total_complaints'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Complaints"),
        opacity=0.8
    ),
    text=[f"{row['month_year']}<br>{row['subdistrict']}<br>{row['total_complaints']:.0f} complaints"
          for _, row in monthly_agg.iterrows()],
    hoverinfo='text'
)])

# Update layout
fig.update_layout(
    title='3D View: Complaints Over Time and Space (Top 10 Subdistricts)',
    scene=dict(
        xaxis=dict(title='Time (Month Index)', backgroundcolor="rgb(230, 230,230)"),
        yaxis=dict(title='Subdistrict Index', backgroundcolor="rgb(230, 230,230)"),
        zaxis=dict(title='Total Complaints', backgroundcolor="rgb(230, 230,230)"),
    ),
    height=700,
    template='plotly_white'
)

fig.show()

# MAE

In [52]:
from catboost import CatBoostRegressor

model = CatBoostRegressor()
model.load_model("./bangkok_traffy_model.cbm")
from sklearn.metrics import mean_absolute_error

# 1. Predict
y_pred = model.predict(X_val)

# 2. Overall MAE
mae_overall = mean_absolute_error(y_val, y_pred)
print(f"Overall MAE: {mae_overall:.4f}")

print("-" * 30)

# 3. MAE per target column
mae_per_col = mean_absolute_error(y_val, y_pred, multioutput='raw_values')

for i, col_name in enumerate(y_val.columns):
    print(f"MAE for {col_name}: {mae_per_col[i]:.4f}")


Overall MAE: 0.4258
------------------------------
MAE for ถนน: 1.3079
MAE for ทางเท้า: 0.8676
MAE for แสงสว่าง: 0.6479
MAE for ความสะอาด: 0.6508
MAE for กีดขวาง: 0.7381
MAE for ท่อระบายน้ำ: 0.4803
MAE for น้ำท่วม: 0.4857
MAE for จราจร: 0.5032
MAE for สายไฟ: 0.3078
MAE for ต้นไม้: 0.4452
MAE for สัตว์จรจัด: 0.2937
MAE for เสียงรบกวน: 0.3984
MAE for คลอง: 0.3006
MAE for PM2.5: 0.1378
MAE for ความปลอดภัย: 0.7254
MAE for ป้าย: 0.5588
MAE for สะพาน: 0.3643
MAE for การเดินทาง: 0.0408
MAE for คนจรจัด: 0.1344
MAE for ป้ายจราจร: 0.0299
MAE for ร้องเรียน: 0.5622
MAE for สอบถาม: 0.1162
MAE for ห้องน้ำ: 0.0367
MAE for เสนอแนะ: 0.0856
